In [15]:
from finlab.crawler import (
    widget, 
    
    crawl_price, 
    crawl_monthly_report, 
    crawl_finance_statement_by_date,
    
    date_range, month_range, season_range
)

import sqlite3
import os
conn = sqlite3.connect(os.path.join('data', "data.db"))

widget(conn, 'price', crawl_price, date_range)
# widget(conn, 'monthly_revenue', crawl_monthly_report, month_range)
# widget(conn, 'finance_statement', crawl_finance_statement_by_date, season_range)

In [204]:
# 找尋多頭且剛突破 20 日線的股票

from finlab.data import Data
import datetime


# 前 150 大股票(0050, 0056) 號碼
stock50Pair = {
        "1101": "台泥", "1102": "亞泥", "1216": "統一", "1301": "台塑", "1303": "南亞", "1326": "台化", "1402": "遠東新", "2002": "中鋼",
        "2105": "正新", "2207": "和泰車"
        , "2301": "光寶科", "2303": "聯電", "2308": "台達電", "2317": "鴻海", "2327": "國巨", "2330": "台積電", "2357": "華碩",
        "2382": "廣達", "2395": "研華", "2408": "南亞科"
        , "2412": "中華電", "2454": "聯發科", "2474": "可成", "2633": "台灣高鐵", "2801": "彰銀", "2823": "中壽", "2880": "華南金",
        "2881": "富邦金", "2882": "國泰金", "2883": "開發金"
        , "2884": "玉山金", "2885": "元大金", "2886": "兆豐金", "2887": "台新金", "2888": "新光金", "2890": "永豐金", "2891": "中信金",
        "2892": "第一金", "2912": "統一超", "3008": "大立光"
        , "3045": "台灣大", "3711": "日月光投控", "4904": "遠傳", "4938": "和碩", "5871": "中租-KY", "5876": "上海商銀", "5880": "合庫金",
        "6505": "台塑化", "9904": "寶成", "9910": "豐泰"}

stock100Pair = {
        "1210": "大成", "1227": "佳格", "1319": "東陽", "1434": "福懋", "1476": "儒鴻", "1477": "聚陽", "1504": "東元", "1536": "和大",
        "1590": "亞德客-KY", "1605": "華新"
        , "1707": "葡萄王", "1717": "長興", "1722": "台肥", "1723": "中碳", "1802": "台玻", "2027": "大成鋼", "2049": "上銀",
        "2059": "川湖", "2101": "南港", "2201": "裕隆"
        , "2204": "中華", "2227": "裕日車", "2231": "為升", "2313": "華通", "2324": "仁寶", "2337": "旺宏", "2344": "華邦電",
        "2345": "智邦", "2347": "聯強", "2352": "佳世達"
        , "2353": "宏碁", "2354": "鴻準", "2356": "英業達", "2360": "致茂", "2371": "大同", "2376": "技嘉", "2377": "微星",
        "2379": "瑞昱", "2383": "台光電", "2385": "群光"
        , "2409": "友達", "2439": "美律", "2449": "京元電子", "2451": "創見", "2492": "華新科", "2498": "宏達電", "2542": "興富發",
        "2606": "裕民", "2610": "華航", "2615": "萬海"
        , "2618": "長榮航", "2809": "京城銀", "2812": "台中銀", "2834": "臺企銀", "2845": "遠東銀", "2867": "三商壽", "2889": "國票金",
        "2903": "遠百", "2915": "潤泰全", "3005": "神基"
        , "3023": "信邦", "3034": "聯詠", "3037": "欣興", "3044": "健鼎", "3231": "緯創", "3406": "玉晶光", "3443": "創意",
        "3481": "群創", "3532": "台勝科", "3533": "嘉澤"
        , "3702": "大聯大", "3706": "神達", "4958": "臻鼎-KY", "5269": "祥碩", "5522": "遠雄", "6176": "瑞儀", "6213": "聯茂",
        "6239": "力成", "6269": "台郡", "6278": "台表科"
        , "6285": "啟碁", "6409": "旭隼", "6415": "矽力-KY", "6456": "GIS-KY", "6592": "和潤企業", "6669": "緯穎", "8046": "南電",
        "8341": "日友", "8454": "富邦媒", "8464": "億豐"
        , "9914": "美利達", "9917": "中保科", "9921": "巨大", "9933": "中鼎", "9941": "裕融", "9945": "潤泰新"
    }
# print(len(stock100List))

allStockList = stock50Pair.copy()
allStockList.update(stock100Pair)
# dict(list(stock50Pair.items()) + list(stock100Pair.items()))
# print(len(allStockList))

# 拿取資料
data = Data()
price60 = data.get('收盤價', 60 ) 
price20 = data.get('收盤價', 20 ) 
price10 = data.get('收盤價', 10 ) 
price5 = data.get('收盤價', 5 ) 
price1 = data.get('收盤價', 1 ) 

valume20 = data.get('成交股數', 20)
valume1 = data.get('成交股數', 1)

# 走多頭的趨勢
is1Large5 = price1[allStockList].mean() > price5[allStockList].mean()
is5Large10 = price5[allStockList].mean() > price10[allStockList].mean()
is10Large20 = price10[allStockList].mean() > price20[allStockList].mean()
is20Large60 = price20[allStockList].mean() > price60[allStockList].mean()

upChoiceStocklist = is1Large5 & is5Large10 & is10Large20 & is20Large60

upStockList = []

for stockId in allStockList:
    if(upChoiceStocklist[stockId] == True):
        upStockList.append(stockId)
#         print("https://goodinfo.tw/StockInfo/StockDetail.asp?STOCK_ID=" + stockId)



# 今日剛突破 20 日線 & 前一天前還沒突破 & 今日成交量比 20 日平均高
less20Before3Day = (price5[allStockList])[-2:-1].mean() < price20[allStockList].mean()
is1Large20 = price1[allStockList].mean() > price20[allStockList].mean() 
is1VolumeLarge20 = valume1[allStockList].mean() > valume20[allStockList].mean() 

breakthrough20StockList = is1Large20 & less20Before3Day & is1VolumeLarge20


print(datetime.date.today())
for stockId in allStockList:
    if(breakthrough20StockList[stockId]):
        print(allStockList[stockId]+"("+ str(price1[stockId].mean())+"): "+"https://goodinfo.tw/StockInfo/StockDetail.asp?STOCK_ID=" + stockId)


2021-05-30
和泰車(564.0): https://goodinfo.tw/StockInfo/StockDetail.asp?STOCK_ID=2207
南亞科(80.6): https://goodinfo.tw/StockInfo/StockDetail.asp?STOCK_ID=2408
新光金(9.93): https://goodinfo.tw/StockInfo/StockDetail.asp?STOCK_ID=2888
大成鋼(44.95): https://goodinfo.tw/StockInfo/StockDetail.asp?STOCK_ID=2027
京城銀(41.3): https://goodinfo.tw/StockInfo/StockDetail.asp?STOCK_ID=2809
臻鼎-KY(97.0): https://goodinfo.tw/StockInfo/StockDetail.asp?STOCK_ID=4958


In [ ]:
# 將結果送到 IM

In [ ]:
# 停利 停損計算